In [6]:
!pip install lxml


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [363]:
import requests
from bs4 import BeautifulSoup
import lxml
import json
import openai
import os
import random
import re
import datetime

In [41]:
url = "https://www.daangn.com/region/"
res= requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text,"html.parser")
value = soup.find_all("option")
cities = []
for i in value:
    city=i.text.strip()
    cities.append(city)
cities = cities[cities.index("동네를 선택하세요")+1:cities.index("한국")]
    

In [42]:
url = f"https://www.daangn.com/region/{city}"
res=requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text,"html.parser")
value  = soup.find_all("option")
villages = []
for i in value:
    village=i.text.strip()
    villages.append(village)
villages=villages[villages.index("동네를 선택하세요")+1:villages.index("한국")]

In [45]:
city_villages=[]
for city in cities[:5]:
    url = f"https://www.daangn.com/region/{city}"
    res=requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text,"html.parser")
    value  = soup.find_all("option")
    villages = []
    for i in value:
        village=i.text.strip()
        villages.append(village)
    villages=villages[villages.index("동네를 선택하세요")+1:villages.index("한국")]
    for village in villages:
        city_villages.append(f"{city}/{village}")

In [46]:
city_village_towns = []

In [47]:
for city_village in city_villages[:5]:
    url = f"https://www.daangn.com/region/{city_village}"
    res=requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text,"html.parser")
    value  = soup.find_all("option")
    towns = []
    for i in value:
        town=i.text.strip()
        towns.append(town)
    towns=towns[towns.index("동을 선택하세요")+1:towns.index("한국")]
    for town in towns:
        city_village_towns.append(f"{city_village}/{town}")

In [314]:
site_list=[]

In [315]:
for city_village_town in city_village_towns:
    url = f"https://www.daangn.com/region/{city_village_town}"
    href_values=[]
    res=requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text,"html.parser")
    a_tag = soup.find_all('a')
    for i in a_tag:
        href_values.append(i['href'])
    href_values = [f"https://www.daangn.com{k}" for k in href_values if 'articles' in k and 'hot' not in k and'www' not in k]
    for k in range(len(href_values)):
        site_list.append(href_values[k])

In [318]:
title_price_description = [{} for i in range(len(site_list[:2000]))]

In [ ]:
for i in range(len(site_list[:2000])):
    try:
        url = site_list[i]
        res = requests.get(url)
        res.raise_for_status()
        soup = BeautifulSoup(res.text,"html.parser")
        
        meta_tag_title = soup.find('meta', {'name': 'twitter:title'})
        title = meta_tag_title['content']

        meta_tag_price = soup.find('meta', {'name': 'twitter:data1'})
        price = meta_tag_price['content']

        meta_tag_description = soup.find('div', {'property': 'schema:description', 'id': 'article-detail'})
        description = meta_tag_description.get_text(strip=True)

        title_price_description[i]["title"] = title
        title_price_description[i]["price"] = price
        title_price_description[i]["description"] = description
        
        print(i)
        
    except TypeError:
        title_price_description[i]["title"] = ""
        title_price_description[i]["price"] = ""
        title_price_description[i]["description"] = ""
        
    except requests.HTTPError:
        title_price_description[i]["title"] = ""
        title_price_description[i]["price"] = ""
        title_price_description[i]["description"] = ""
        

In [322]:
import json

나눔 지우고, description 긴거 보자

In [323]:
result = []
for i in range(len(title_price_description)):
    if 100<len(title_price_description[i]['description'])<500 and '나눔' not in title_price_description[i]['price']:
        result.append(title_price_description[i])
len(result)

760

In [ ]:
for i in range(len(result)):
    if '나눔' in result[i]["title"]:
        print('제목',i)
    elif '나눔' in result[i]["description"]:
        print('설명',i)

In [326]:
new_file_path = f'/opt/crong.json'  
with open(new_file_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [340]:
with open(new_file_path,'r',encoding='utf-8') as f:
    json_data=f.read()
    data=json.loads(json_data)

In [342]:
for i in range(len(data)):
    data[i]["result"]=""
    data[i]["events"]=[]

In [343]:
data

[{'title': '버버리 바람막이 ',
  'price': '160,000원',
  'description': '버버리 바람막이 100~110 입으시분 잘 맞아요상태는 오염 없고 최상급 깨끗하고 좋아요여행갈때 주말 나들이에 필수로걸치거나 가방안에 넣고,다닐 수 있는. 바람막이 입니다버버리, 노바 체크가 예쁘게 조화되서 넘 고급스럽고 예뻐요실지보면 더 예뻐요반품,환불 어렵습니다',
  'result': '',
  'events': []},
 {'title': '루이비통 스피드',
  'price': '250,000원',
  'description': '남편이 선물해준 가방이구요.구매시기는정확하게 기억이 나지 않네요.넘 좋아서 열심히 들고 다녔는데지금은 제가 일을 해서 큰가방이 필요해서들고 다니지 않아서 판매 하려고 합니다.사용감 있어요.',
  'result': '',
  'events': []},
 {'title': '루이비통 모노그램 체인팔찌 실버 판매합니다',
  'price': '300,000원',
  'description': '루이비통에서 한정판으로 출시한 "모노그램 체인 팔찌". (전 매장 품절 및 Reorder 없는 제품으로 중고 제품 P가 계속 상승중)실물 이뻐용가격은 매우 저렴하게 판매합니다생활기스 조금 있습니다. (사진참조)구성품은 없고 단품이에요',
  'result': '',
  'events': []},
 {'title': '알라이아 alaia 펀칭백',
  'price': '920,000원',
  'description': '알라이아 펀칭백 입니다몇년 전 해외 구매입니다더스트백 등 구성품 없습니다바닥사이즈는 약 16.5*16.5높이 22센치 입니다보관상 문제인지 한쪽은 괜찮은데 다른 한쪽면이 주름이 많이 잡혀 무너짐이 있어서 사진 자세히 찍었습니다대치동 직거래만 가능합니다개인중고거래 특성상 교환환불 불가입니다신중하게 보시고 거래부탁드립니다질문만 하시고 사라지시는 분 비매너분 차단합니다',
  'result': '',
  'event